In [1]:
!pip install -e .

Obtaining file:///Users/Ashley/Documents/GitHub/ie_pandas
  Found existing installation: ie-pandas 0.1.dev0
    Uninstalling ie-pandas-0.1.dev0:
      Successfully uninstalled ie-pandas-0.1.dev0
  Running setup.py develop for ie-pandas


In [2]:
from ie_pandas import DataFrame

Done __init__.py


In [3]:
dic = {'col_1': [3, 2, 1.0, 0],
       'col_2': ['as', 'bs', 'cs', 'ds'],
       'col_3': [3, 2, 1, 0],
       'col_4': ['a', 'b', 'c', 'd']}
dic

{'col_1': [3, 2, 1.0, 0],
 'col_2': ['as', 'bs', 'cs', 'ds'],
 'col_3': [3, 2, 1, 0],
 'col_4': ['a', 'b', 'c', 'd']}

In [4]:
dic_pandas = DataFrame(dic, rowindex = ['R1', 'R2', 'R3', 'R4'], colindex=['A', 'B', 'C', 'D'])
dic_pandas

      A   B  C  D
R1    3  as  3  a
R2    2  bs  2  b
R3  1.0  cs  1  c
R4    0  ds  0  d

In [5]:
mylist = [[1, 2, 3],
          [1.0, 2.3, 4.5],
          ['a', 'b', 'c'],
          [True, False, True]]

In [6]:
import numpy as np
npmylist = np.array(mylist)
print(type(npmylist))
npmylist

<class 'numpy.ndarray'>


array([['1', '2', '3'],
       ['1.0', '2.3', '4.5'],
       ['a', 'b', 'c'],
       ['True', 'False', 'True']], dtype='<U32')

In [7]:
df_npmylist = DataFrame(npmylist)
df_npmylist

   0    1  2      3
0  1  1.0  a   True
1  2  2.3  b  False
2  3  4.5  c   True

In [8]:
df = DataFrame(mylist, axis = 0)
df

   0    1  2      3
0  1  1.0  a   True
1  2  2.3  b  False
2  3  4.5  c   True

In [9]:
df.get_row(0)

[1, 1.0, 'a', True]

In [10]:
df = DataFrame(mylist, colindex = ['C1', 'C2', 'C3', 'C4'])
df

   C1   C2  C3     C4
0   1  1.0   a   True
1   2  2.3   b  False
2   3  4.5   c   True

In [11]:
df = DataFrame(mylist, rowindex = ['R1', 'R2', 'R3'])
df

    0    1  2      3
R1  1  1.0  a   True
R2  2  2.3  b  False
R3  3  4.5  c   True

In [12]:
df = DataFrame(mylist, colindex = ['C1', 'C2', 'C3', 'C4'], rowindex = ['R1', 'R2', 'R3'])
df

    C1   C2  C3     C4
R1   1  1.0   a   True
R2   2  2.3   b  False
R3   3  4.5   c   True

In [13]:
df[3]

array([ True, False,  True])

In [14]:
## df[df['colindex'][1]]

In [15]:
DataFrame(10)

Exception: The input should be a list, a list of lists, a dictionary, or a numpy array. Now, it is a <class 'int'>

In [ ]:
type(df)

In [ ]:
# df.pretty(['b','a'])